In [35]:
import requests
import json

predictionUrl = ""
predictionKey = ""

def predict(image):
  headers = {'content-type' : 'application/octet-stream','Prediction-Key' : predictionKey}
  request = requests.post(predictionUrl,data=open(image,"rb"),headers=headers)

  probability = json.loads(request.content.decode('utf8').replace("'", '"'))['predictions'][0]['probability']
  tag = json.loads(request.content.decode('utf8').replace("'", '"'))['predictions'][0]['tagName']
  box = json.loads(request.content.decode('utf8').replace("'", '"'))['predictions'][0]['boundingBox']

  return probability, tag, box

print(predict("sample.jpg"))

(0.92584467, 'lady_beetle', {'left': 0.093494415, 'top': 0.2742829, 'width': 0.755642, 'height': 0.55387485})
